# data/KBO 폴더에 있는 외부 데이터와 input 폴더에 있는 데이터를 불러온 후, 
# 외부 데이터에 없는 SF열을 추가하여 데이터를 data/sf_data 폴더에 저장함.

# 데이터 불러오기, 변환, 병합

In [1]:
import pandas as pd
import numpy as np

import os 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('font', family='NanumBarunGothic') 
plt.style.use(['ggplot'])

import warnings
warnings.filterwarnings(action='ignore') 

# 사용자 운영체제 확인
import platform
platform.system()

# 운영체제별 한글 폰트 설정
if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')

plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정


# 글씨 선명하게 출력하는 설정
%config InlineBackend.figure_format = 'retina'

In [2]:
train_path = '../input/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_'
# HTS (2018 ~ 2021)
HTS2018 = pd.read_csv(train_path + 'HTS_2018.csv', encoding = 'cp949')
HTS2019 = pd.read_csv(train_path + 'HTS_2019.csv', encoding = 'cp949')
HTS2020 = pd.read_csv(train_path + 'HTS_2020.csv', encoding = 'cp949')
HTS2021 = pd.read_csv(train_path + 'HTS_2021.csv', encoding = 'cp949')
# 선수 (2018 ~ 2021)
Player2018 = pd.read_csv(train_path + '선수_2018.csv', encoding = 'cp949')
Player2019 = pd.read_csv(train_path + '선수_2019.csv', encoding = 'cp949')
Player2020 = pd.read_csv(train_path + '선수_2020.csv', encoding = 'cp949')
Player2021 = pd.read_csv(train_path + '선수_2021.csv', encoding = 'cp949')
# 타자 기본 (2018 ~ 2021)
Hitter2018 = pd.read_csv(train_path + '타자 기본_2018.csv', encoding = 'cp949')
Hitter2019 = pd.read_csv(train_path + '타자 기본_2019.csv', encoding = 'cp949')
Hitter2020 = pd.read_csv(train_path + '타자 기본_2020.csv', encoding = 'cp949')
Hitter2021 = pd.read_csv(train_path + '타자 기본_2021.csv', encoding = 'cp949')
# 팀 (single)
Team = pd.read_csv(train_path + '팀.csv', encoding = 'cp949')
# 경기일정 (single)
Calender = pd.read_csv(train_path + '경기일정_2021.csv', encoding = 'cp949')

In [3]:
# PIT_ID가 중복되는 Record가 224개 있음. --> PIT_ID가 시간-분-초로 이루어져 있어 같은 날에 시분초가 다 똑같으면 PIT_ID가 중복되어버림. 
# 해결책 : PIT_ID의 각 열을 year, month, day, hour, minute, second로 쪼개어 독립 열로 분리 후 PIT_ID 열 삭제. 
# 주의 : PIT_ID는 어쨌든 경기 날짜와 시간 정보를 담고 있기 때문에 시각화 및 분석을 위해 일단 남겨두는 방법을 택함. 
duplicated_PIT_ID = HTS2018[HTS2018.PIT_ID.duplicated()].PIT_ID.to_list()
display(len(duplicated_PIT_ID))
for cnt, ID in enumerate(duplicated_PIT_ID):
    display(HTS2018.query('PIT_ID == @ID'))
    if cnt == 5: # 더 보고 싶으면 숫자 바꿀 수 있음.
        break

224

,GYEAR,G_ID,PIT_ID,PCODE,T_ID,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,STADIUM
10,2018,20180324HHWO0,180324_145431,62797,HH,3,63.08,-34.8,땅볼아웃,145.41,고척
107,2018,20180324LGNC0,180324_145431,62931,NC,3,108.48,22.9,1루타,145.78,마산


,GYEAR,G_ID,PIT_ID,PCODE,T_ID,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,STADIUM
758,2018,20180328KTSK0,180328_193206,74215,KT,4,171.67,-3.8,1루타,129.06,문학
813,2018,20180328LGWO0,180328_193206,67341,WO,3,92.80,52.0,플라이,121.20,고척


,GYEAR,G_ID,PIT_ID,PCODE,T_ID,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,STADIUM
723,2018,20180328HHNC0,180328_203348,60100,HH,7,148.19,-4.7,땅볼아웃,140.56,마산
874,2018,20180328LTOB0,180328_203348,75334,OB,6,157.73,15.9,직선타,148.11,잠실


,GYEAR,G_ID,PIT_ID,PCODE,T_ID,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,STADIUM
963,2018,20180329HHNC0,180329_195925,78813,NC,4,139.88,35.7,플라이,132.12,마산
1023,2018,20180329KTSK0,180329_195925,76812,SK,5,96.81,31.5,플라이,132.28,문학


,GYEAR,G_ID,PIT_ID,PCODE,T_ID,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,STADIUM
1032,2018,20180329KTSK0,180329_202200,62895,SK,6,136.18,26.4,플라이,140.18,문학
1120,2018,20180329LTOB0,180329_202200,75334,OB,6,151.90,13.0,1루타,116.63,잠실


,GYEAR,G_ID,PIT_ID,PCODE,T_ID,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,STADIUM
1202,2018,20180330HTLG0,180330_204818,79192,LG,6,115.24,14.5,2루타,138.90,잠실
1387,2018,20180330WOSS0,180330_204818,64610,SS,7,142.80,-0.2,1루타,140.89,대구


In [4]:
# 결측치 확인
display(Player2020[Player2020['MONEY'].isna()])
Player2021[Player2021['MONEY'].isna()]

# 무조건 만원 아니면 달러로 구분되어있음. 
# Player2018[Player2018['MONEY'].str.contains('만원')].shape[0] + Player2018[Player2020['MONEY'].str.contains('달러')].shape[0] == Player2018.shape[0]

,GYEAR,PCODE,NAME,T_ID,POSITION,AGE_VA,MONEY
64,2020,50802,화이트,SK,내,29,NaN


,GYEAR,PCODE,NAME,T_ID,POSITION,AGE_VA,MONEY
442,2021,67610,김석환,HT,내,22,NaN
457,2021,68069,고명성,KT,내,22,NaN
529,2021,69645,장지수,HT,투,21,NaN
548,2021,71752,김태균,HH,내,39,NaN


In [5]:
class DataTransform:
  def __init__(self, HTS_data, Player_data, Hitter_data, Team_data, Calender_data):
    self.HTS_data = HTS_data
    self.Player_data = Player_data
    self.Hitter_data = Hitter_data
    self.Team_data = Team_data
    self.Calender_data = Calender_data


  def MergingData(self):
      def StrToDate(self):
        self.HTS_data['PIT_ID'] = pd.to_datetime(self.HTS_data['PIT_ID'],format = '%y%m%d_%H%M%S')
        self.HTS_data['MONTH'] = self.HTS_data['PIT_ID'].astype('datetime64')
        self.HTS_data['YEAR'] = self.HTS_data['PIT_ID'].dt.year
        self.HTS_data['MONTH'] = self.HTS_data['PIT_ID'].dt.month
        self.HTS_data['DAY'] = self.HTS_data['PIT_ID'].dt.day
        self.HTS_data['HOUR'] = self.HTS_data['PIT_ID'].dt.hour
        self.HTS_data['MINUTE'] = self.HTS_data['PIT_ID'].dt.minute
        self.HTS_data['SECOND'] = self.HTS_data['PIT_ID'].dt.second
        # 'PIT_ID'는 시각화룰 위해 지우지 않음 
        # HTS_data['PIT_ID'] = HTS_data.index
        return self.HTS_data

      # 20년,21년 Player 데이터에 결측값 1개,4개 확인 - 삭제 
      def DropNa(data):
        data.dropna(inplace = True)
        return data
      # 만원, 달러 숫자로 변환   
      def MoneyExchange(MONEY):
        if MONEY.find('만원') != -1:
          return int(MONEY.split('만')[0]) * 10000
        else:
          return int(MONEY.split('달')[0]) * 1146 

      def Make_OBP_OPS(self):
        self.Hitter_data['OBP'] = (self.Hitter_data['HIT'] + self.Hitter_data['BB'] + self.Hitter_data['HP']) /(self.Hitter_data['AB'] + self.Hitter_data['BB'] + self.Hitter_data['HP'] + self.Hitter_data['SF'])
        self.Hitter_data['OPS'] = self.Hitter_data['OBP'] + self.Hitter_data['SLG']
        return self.Hitter_data
        
      def Make_7_Class(self):
        include_result = ['1루타','2루타','3루타','홈런','플라이','파울플라이','직선타']
        self.HTS_data = self.HTS_data.query('HIT_RESULT in @include_result')
        return self.HTS_data
        

      DropNa(self.Player_data)
      StrToDate(self)
      Make_OBP_OPS(self)
      self.Player_data['MONEY'] = self.Player_data['MONEY'].apply(MoneyExchange)
      new_df = pd.merge(self.HTS_data, self.Player_data, on = ['PCODE']).merge(self.Hitter_data, on = ['PCODE'])
      new_df.drop(['GYEAR','GYEAR_y','T_ID_y'],axis = 1, inplace = True)
      new_df.rename(columns = {'GYEAR_x' : 'GYEAR', 'T_ID_x' : 'T_ID'}, inplace = True)
      new_df['BABIP'] = (new_df['HIT'] -  new_df['HR']) / (new_df['AB'] - new_df['KK'] - new_df['HR'] + new_df['SF'])
      new_df = new_df.sort_values(by = ['BABIP'], ascending= False)

      return new_df

In [6]:
a = DataTransform(HTS2018, Player2018, Hitter2018, Team, Calender)
b = DataTransform(HTS2019, Player2019, Hitter2019, Team, Calender)
c = DataTransform(HTS2020, Player2020, Hitter2020, Team, Calender)
d = DataTransform(HTS2021, Player2021, Hitter2021, Team, Calender)
df_2018 = a.MergingData()
df_2019 = b.MergingData()
df_2020 = c.MergingData()
df_2021 = d.MergingData()

df = pd.concat([df_2018, df_2019, df_2020, df_2021], axis = 0).reset_index(drop = True)

In [7]:
df1 = df.query('PCODE == [76232, 68050, 75847, 67341, 79192, 78224, 78513, 76290, 79215, 67872]')

In [8]:
df1['G_ID'] = df1['G_ID'].str[:8]
df1 = df1.sort_values(by=['G_ID'], ascending=True).reset_index(drop=True, inplace=False)

# 외부데이터 추가 (볼넷 사구 추가위함)

In [9]:
#데이터 불러오기
playerList = ['양의지', '강백호', '최정', '이정후', '채은성', '김재환', '전준우', '김현수', '박건우', '로맥']
playerCode = ['76232', '68050', '75847', '67341', '79192', '78224', '78513', '76290', '79215', '67872']

for player in playerList:
    code = playerCode[playerList.index(player)] #playerCode와 playerList의 인덱스번호를 이용해서 맞는 짝을 찾아줌
    for year in range(2018,2022,1):
        train_path = '../data/KBO_data' # 데이터 저장 경로 
        globals()[f'df{year}_{code}'] = pd.read_csv(train_path + f'\{player} {year}.csv', encoding = 'cp949', index_col = 0)
        globals()[f'df{year}_{code}']['YEAR'] = f'{year}' #년도 구분추가
        globals()[f'df{year}_{code}'] = globals()[f'df{year}_{code}'].reset_index()

#### 해당 연도,월 선수코드에 해당하는 것 중에 희생플라이인 행만 뽑아서 len을 통해 SF열을 생성

In [10]:
dfsf = pd.DataFrame({'YEAR' : [0],'PCODE' : [0], 'MONTH' : [0], 'SF' : [0]}) 

for year in df1['YEAR'].unique():
    df2 = df1.query('GYEAR == @year')
    for month in df2['MONTH'].unique():
        for pcode in df2['PCODE'].unique():
            df3 = df2.query('MONTH == @month').query('PCODE == @pcode')
            df4 = df3.query('HIT_RESULT=="희생플라이"')
            if df4.shape[0] == 0:
                SF = 0
            else : 
                SF = len(df4)
            dfsf.loc[len(dfsf)] = [year,pcode,month,SF]

dfsf = dfsf.loc[1:].reset_index(drop = True)
dfsf = dfsf.sort_values(by = ['YEAR','PCODE','MONTH'])

In [11]:
#dfst라는 sf 데이터프레임을 한번에 생성하고 선수코드, 년도별로 필요한 sf를 뽑아씀
for year in ['2018','2019','2020','2021']:
    for player in ['76232', '68050', '75847', '67341', '79192', '78224', '78513', '76290', '79215', '67872']:
        dfsf.query('YEAR == @year and PCODE == @player').reset_index()
        f'df{year}_{player} = df{year}_{player}.reset_index()'
        f"df{year}_{player}['SF'] = dfsf.query('YEAR == @year and PCODE == @player')['SF']"

# 각 데이터프레임에 SF를 추가

In [13]:
#선수별로 SF 열을 추가해줌
years = [2018,2019,2020,2021]
playerCode = ['76232', '68050', '75847', '67341', '79192', '78224', '78513', '76290', '79215', '67872']
playerList = ['양의지', '강백호', '최정', '이정후', '채은성', '김재환', '전준우', '김현수', '박건우', '로맥']

for code in playerCode:
    player = playerList[playerCode.index(code)] #playerCode와 playerList의 인덱스번호를 이용해서 맞는 짝을 찾아줌
    for year in years: #년도별로 진행
        comment = f'YEAR == {year} and PCODE == {code}'
        path = '../data/sf_data' #new_data라는 폴더를 만들어 데이터를 그 폴더에 저장
        
        globals()[f'df{year}_{code}']['SF'] = dfsf.query(comment).reset_index()['SF']
        globals()[f'df{year}_{code}'].to_csv(f'{path}/{player} {year}_sf.csv')

# END